In [1]:
import pandas as pd
import yfinance as yf

In [2]:
#read super_winners.csv
super_winners = pd.read_csv('super_winners.csv')
median_cagrs = pd.read_csv('median_cagrs.csv')
industries = pd.read_csv('Companies Industries.csv')

In [3]:
#reset all the indexes
super_winners = super_winners.reset_index(drop=True)
median_cagrs = median_cagrs.reset_index(drop=True)
industries = industries.reset_index(drop=True)

In [4]:
#order median_cagrs by 30YR_median
median_cagrs = median_cagrs.sort_values(by=['30YR_median'], ascending=False)

In [5]:
#get all the super winners with at least 14 30YRWins
oldWinners = super_winners[super_winners['30YRWins'] >= 14]
len(oldWinners)

85

In [6]:
highCagrs = median_cagrs[median_cagrs['30YR_median'] >= 12]
len(highCagrs)

86

In [7]:
#merge the two dataframes
oldAndHigh = pd.merge(oldWinners, highCagrs, on='Ticker', how='inner')
len(oldAndHigh)

47

In [8]:
#order oldAndHigh by 30YR_median
oldAndHigh = oldAndHigh.sort_values(by=['30YR_median'], ascending=False)
oldAndHigh = oldAndHigh.reset_index(drop=True)
#drop Unnamed: 0_x and Unnamed: 0_y
oldAndHigh = oldAndHigh.drop(columns=['Unnamed: 0_x', 'Unnamed: 0_y'])

In [9]:
#add industry column to oldAndHigh
oldAndHigh = pd.merge(oldAndHigh, industries, on='Ticker', how='inner')

In [10]:
oldAndHigh.to_csv('oldAndHigh.csv', index=False)

In [11]:
def get_company_info(ticker_symbol):
    try:
        ticker = yf.Ticker(ticker_symbol)
        company_info = ticker.info
        company_name = company_info['longName']
        sector = company_info.get('sector', 'N/A')
        industry = company_info.get('industry', 'N/A')
        print('Name:', company_name, 'Sector:', sector, 'Industry:', industry)
        return company_name, sector, industry
    except Exception as e:
        print('Error:', str(e))
        return None, None, None

In [ ]:
#create an empty dataframe with the columns Ticker, Company, Sector, and Country
companies = pd.DataFrame(columns=['Ticker', 'Company', 'Sector', 'Industry', 'Country'])
for i in range(len(oldAndHigh)):
    ticker = oldAndHigh['Ticker'][i]
    name, sector, industry = get_company_info(ticker)
    country = oldAndHigh['Country'][i]
    companies.loc[i] = [ticker, name, sector, industry, country]
companies.to_csv('companies.csv', index=False)